Pickle Data 불러오기

In [1]:
import pickle

with open("raw_documents.pkl", "rb") as f:
    data_list = pickle.load(f)

print(len(data_list))  # 문서 개수 확인


53209


불러온 데이터 확인

In [2]:
from collections import Counter

case_counter = Counter(doc["__case_type"] for doc in data_list)  #모든 문서에서 case_type값을 하나씩 센다.
print("사건유형별 문서 수:")
for k, v in case_counter.items(): #items()는 딕셔너리같은 자료구조에서 k,v 쌍을 튜플로 꺼내주는 함수
    print(f"{k}: {v}건")

사건유형별 문서 수:
08.특허.저작권: 2407건
10.개인정보.ICT: 1603건
06.기업: 1712건
09.금융조세: 2433건
01.민사: 13623건
05.행정: 17070건
02.가사: 1655건
07.근로자: 2408건
04.형사B(일반형): 5491건
03.형사A(생활형): 4807건


## 청킹

data_list에 있는 각 문서의 "판례내용"을 500자 단위로 Chunking하고, 메타데이터와 함께 리스트로 저장하기

Chunking 함수 

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_law_document(doc, chunk_size=500, chunk_overlap=50):   #청크사이즈 : 500, 오버랩 : 50   // 하나의 문서를 매개변수로 받아온다.
    text = doc.get("판례내용", "") #doc은 결국에 하나의 딕셔너리다. get()으로 "판례내용"이라는 key의 value값을 받아온다. => 만약에 key가 없으면 기본값 반환
    if not text:
        return []

    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = splitter.split_text(text) #판례내용만 청킹한다.  // chunks는 여러개의 chunk들이 담겨있는 리스트

    meta_fields = [       # 각 chunk에 같이 붙여줄 메타데이터 필드 이름 리스트
    "판례일련번호", "사건명", "사건번호", "선고일자", "법원명", "법원종류코드", "사건종류명", "사건종류코드", "판결유형", "선고", "판례상세링크", "판시사항", "판결요지", "참조조문", "참조판례"
    ]

    return [
        {
            "text": chunk,
            "chunk_id": i,
            **{k: doc.get(k) for k in meta_fields}, # **는 딕셔너리 언패킹 // meta_fields 리스트에 있는 key값들에 해당하는 것들만 뽑아서 다시 딕셔너리에 추가
            "__file_path": doc.get("__file_path"),
        }
        for i, chunk in enumerate(chunks)  #모든 청크들에 대해서 반복
    ] # 결국 딕셔너리들을 원소로 같는 list를 return 한다.


In [5]:
all_chunks = []

for doc in data_list:
    chunks = chunk_law_document(doc) #하나의 문서를 매개변수로 전달
    all_chunks.extend(chunks) # extend()는 리스트에 요소들을 펼쳐서 추가(병합)

print(f"총 chunk 수: {len(all_chunks)}")


총 chunk 수: 408330


데이터 확인

In [6]:
print(all_chunks[0])


{'text': '【원고,상고인겸피상고인】 원고 (소송대리인 변호사 이익우)\n【피고,상고인】 창도종합건설 주식회사 (소송대리인 변호사 이인철)\n【피고,피상고인겸상고인】 건설공제조합 (소송대리인 동남합동법률사무소 담당변호사 전정구)\n【원심판결】 서울고법 1996. 5. 2. 선고 94나17316 판결\n【주문】\n원심판결 중 피고 건설공제조합에 대한 원고 패소 부분을 파기하고, 이 부분 사건을 서울고등법원에 환송한다. 피고들의 상고를 모두 기각하고, 이 부분 상고비용은 피고들의 부담으로 한다.\n【이유】\n상고이유를 본다.\n1. 피고 창도종합건설 주식회사(이하 피고 회사라 한다)의 상고이유 제1점에 대하여', 'chunk_id': 0, '판례일련번호': 195152, '사건명': '공사금반환', '사건번호': '96다23306', '선고일자': '1999-03-26', '법원명': '대법원', '법원종류코드': None, '사건종류명': '민사', '사건종류코드': 400101, '판결유형': '판결', '선고': '선고', '판례상세링크': '/DRF/lawService.do?OC=pwshoot&target=prec&ID=195152&type=HTML&mobileYn=', '판시사항': '【판시사항】\n[1] 도급인과 수급인이 계약보증금의 한도에 관한 약정만 하고 그 보증금 상당액을 위약벌이나 손해배상액의 예정으로 하는 특약이 없는 상태에서 건설공제조합이 계약보증을 한 경우, 수급인의 귀책사유로 공사도급계약이 해제된 사유만으로 도급인이 건설공제조합에게 보증금 전액을 청구할 수 있는지 여부(소극)\n[2] 구 건설공제조합법 제2조 제6호에 정한 계약보증을 한 건설공제조합의 책임범위\n[3] 공사도급계약상 지체상금 약정의 적용범위를 정하는 판단 기준 및 그 산정방법', '판결요지': '【판결요지】\n[1] 도급인과 수급인 사이에 계약보증금의 한도에 관한 약정이 있었을 뿐, 보증금에 해당하는 금액을 위약벌 내지 제재금이나 손해배상액의 예정으로 하는 특약이 있

In [7]:
print(all_chunks[1])

{'text': '1. 피고 창도종합건설 주식회사(이하 피고 회사라 한다)의 상고이유 제1점에 대하여\n기록에 비추어 살펴보면, 원심이 그 판시와 같은 사실을 인정한 다음, 피고 회사가 14일 이상 공사를 중단하였음을 이유로 한 원고의 이 사건 공사도급계약 해제는 적법하다고 판단하고, 그 적법성을 다투는 피고 회사의 주장들을 배척한 조치는 옳다고 수긍이 가고, 거기에 채증법칙을 위반한 사실오인이나 심리미진의 위법이 있다고 할 수 없으므로, 이에 대한 상고이유의 주장은 받아들일 수 없다.\n2. 피고 회사의 상고이유 제2점에 대하여', 'chunk_id': 1, '판례일련번호': 195152, '사건명': '공사금반환', '사건번호': '96다23306', '선고일자': '1999-03-26', '법원명': '대법원', '법원종류코드': None, '사건종류명': '민사', '사건종류코드': 400101, '판결유형': '판결', '선고': '선고', '판례상세링크': '/DRF/lawService.do?OC=pwshoot&target=prec&ID=195152&type=HTML&mobileYn=', '판시사항': '【판시사항】\n[1] 도급인과 수급인이 계약보증금의 한도에 관한 약정만 하고 그 보증금 상당액을 위약벌이나 손해배상액의 예정으로 하는 특약이 없는 상태에서 건설공제조합이 계약보증을 한 경우, 수급인의 귀책사유로 공사도급계약이 해제된 사유만으로 도급인이 건설공제조합에게 보증금 전액을 청구할 수 있는지 여부(소극)\n[2] 구 건설공제조합법 제2조 제6호에 정한 계약보증을 한 건설공제조합의 책임범위\n[3] 공사도급계약상 지체상금 약정의 적용범위를 정하는 판단 기준 및 그 산정방법', '판결요지': '【판결요지】\n[1] 도급인과 수급인 사이에 계약보증금의 한도에 관한 약정이 있었을 뿐, 보증금에 해당하는 금액을 위약벌 내지 제재금이나 손해배상액의 예정으로 하는 특약이 있었다고는 인정되지 아니하는 경우, 도급인이 수급인의 채무불이행을 이유로 보증인에 대

In [8]:
from collections import Counter

# 사건종류별 chunk 수 세기
type_count = Counter(chunk["사건종류명"] for chunk in all_chunks)
print("사건 종류별 chunk 수:")
for k, v in type_count.items():
    print(f"{k}: {v}개")


사건 종류별 chunk 수:
민사: 170326개
일반행정: 56927개
기업_공정위: 1631개
세무: 28243개
기업_공동행위: 2836개
형사: 112706개
행정_시정명령: 217개
기업_해고: 426개
제조물_책임_민사: 97개
근로_산재: 714개
행정_정보공개: 1983개
특허: 16720개
가사_이혼: 3749개
행정: 1348개
행정강제: 41개
가사: 7014개
가사_상속: 2161개
근로_임금: 536개
형사_교통: 280개
형사_폭행: 375개


저장 

In [9]:
import pickle

with open("chunked_documents.pkl", "wb") as f:
    pickle.dump(all_chunks, f)

print("✅ chunked_documents.pkl 파일로 저장 완료!")



✅ chunked_documents.pkl 파일로 저장 완료!
